In [393]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [394]:
x_min = -np.pi / 2
x_max = np.pi / 2

t_max = 0.15

In [395]:
def FTCS(ratio:float, N:int):

    x = np.linspace(x_min, x_max, N)
    dx = x[1] - x[0]
    dt = (dx**2) / ratio
    # Initial condition
    u = np.cos(3.0*x)

    # Boundary condition
    u[0] = u[-1] = 0

    t = 0.0
    while t < t_max:
        
        if t > (t_max - dt):
            # This step ensures that the final step updates exactly to t_max
            dt = t_max - t
        t = t + dt
        
        r = dt / (dx**2)
        u_cache = u.copy()
        for i in range(1, N-1):
            # FTCS scheme
            u_cache[i] = u[i] + r * (u[i+1] - 2 * u[i] + u[i-1])
        u = u_cache
    
    # To compare with exactly solution at a given time, exact solution must be calculated directly from this given time
    v = np.exp(-9.0 * t_max) * np.cos(3.0 * x)
    error = np.abs(u - v)
    L2_norm = np.sqrt(np.sum(error**2) * dx)
    Linf_norm = np.max(error)

    return u, v, dx, L2_norm, Linf_norm

In [396]:
N_list = [50, 100, 200, 400]
result = []
for N in N_list:
    _, _, dx, L2_error, Linf_error = FTCS(2.0, N)
    result.append([N, dx, L2_error, Linf_error])

In [397]:
df_2 = pd.DataFrame(result, columns=["N", "dx", "L_2 error", "L_inf order"])

In [398]:
L2_order = []
Linf_order = []
for i in range(1, 4):
    L2 = np.log2(result[i][2] / result[i-1][2]) / np.log2(result[i][1] / result[i-1][1])
    Linf = np.log2(result[i][3] / result[i-1][3]) / np.log2(result[i][1] / result[i-1][1])
    L2_order.append(L2)
    Linf_order.append(Linf)

L2_order.insert(0, 'NA')
Linf_order.insert(0, 'NA')
df_2.insert(loc=3, column='L_2 Convergence order', value=L2_order)
df_2.insert(loc=5, column='L_inf Convergence order', value=Linf_order)

In [399]:
print(df_2)

     N        dx  L_2 error L_2 Convergence order  L_inf order  \
0   50  0.064114   0.002719                    NA     0.002168   
1  100  0.031733   0.000663              2.006186     0.000529   
2  200  0.015787   0.000164                2.0013     0.000131   
3  400  0.007874   0.000041              2.000068     0.000033   

  L_inf Convergence order  
0                      NA  
1                2.007066  
2                1.999721  
3                2.000123  


In [400]:
result = []
for N in N_list:
    _, _, dx, L2_error, Linf_error = FTCS(6.0, N)
    result.append([N, dx, L2_error, Linf_error])

In [401]:
df_6 = pd.DataFrame(result, columns=["N", "dx", "L_2 error", "L_inf order"])

In [402]:
L2_order = []
Linf_order = []
for i in range(1, 4):
    L2 = np.log2(result[i][2] / result[i-1][2]) / np.log2(result[i][1] / result[i-1][1])
    Linf = np.log2(result[i][3] / result[i-1][3]) / np.log2(result[i][1] / result[i-1][1])
    L2_order.append(L2)
    Linf_order.append(Linf)

L2_order.insert(0, 'NA')
Linf_order.insert(0, 'NA')
df_6.insert(loc=3, column='L_2 Convergence order', value=L2_order)
df_6.insert(loc=5, column='L_inf Convergence order', value=Linf_order)

In [403]:
print(df_6)

     N        dx     L_2 error L_2 Convergence order   L_inf order  \
0   50  0.064114  1.437320e-06                    NA  1.146226e-06   
1  100  0.031733  1.373817e-07              3.338225  1.094906e-07   
2  200  0.015787  7.411777e-09              4.181833  5.913558e-09   
3  400  0.007874  5.871646e-10              3.644787  4.684562e-10   

  L_inf Convergence order  
0                      NA  
1                3.339106  
2                4.180254  
3                3.644845  


## Explanation of the result

When $dt=dx^2/2$, we get the expected result that the convergence order is 2. Since the $O(\Delta x^2)$ term in the truncation error is 

$\frac{1}{2}v_{tt}\Delta t - \frac{1}{12} v_{xxxx} \Delta x^2$,

when $dt=dx^2/6$, this term also vanish, thus give $O(\Delta x^4)$ accuracy.
However, the order is not exactly 4, and when I discard the if condition and $dt=t_{max}-t$ step, at the same time calculate the exact solution with t from the last step, the order will be closer to 4. Firstly, this step is necessary because in reality we shouldn't calculate from the last t instead of a given t_max. Secondly, this finding suggests that even we ensures we are comparing numerical and exact solutions at the same time, the step size of the last step matters. If we modify the last time step, the relation $dt=dx^2/6$ is not satisfied at the last step, resulting in a truncation error slightly larger than 
$O(\Delta x^4)$ but still significantly better than $O(\Delta x^2)$.